In [11]:
from PIL import Image,ImageChops,ImageDraw
import numpy as np
import os

In [4]:
def square_crop_in_center(img):
    '''
    Crop the image to the biggest centered square
    Takes an image, returns an image
    '''
    width, height = img.size  # Get dimensions
    new_width = min(width, height)
    new_height = min(width, height)

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2

    # Crop the center of the image
    return img.crop((left, top, right, bottom))

In [8]:
def image_to_circle(img):
    '''
    From any image format, provides a circle image (centered)
    Takes a PIL image, returns a PIL image
    '''
    img = img.convert("RGB")
    # Make img as a square if not
    h, w = img.size
    if h != w:
        img = square_crop_in_center(img)

    # Open the input image as numpy array, convert to RGB
    npImage = np.array(img)
    h, w = img.size
    radius = min(h, w)

    # Create same size alpha layer with circle
    alpha = Image.new('L', (h,w),0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([0, 0, radius, radius], 0, 360, fill=255)

    # Convert alpha Image to numpy array
    npAlpha=np.array(alpha)

    # Add alpha layer to RGB
    npImage=np.dstack((npImage,npAlpha))

    # Save with alpha
    return Image.fromarray(npImage)

In [9]:
def image_to_vignette(img,overlay):

    img = image_to_circle(img)

    img = img.convert("RGBA")
    overlay = overlay.convert("RGBA")
    #print(f"Initial size is: image = {img.size} and overlay = {overlay.size}")

    # Coef used to oversize the overlay, in order to keep image picture quality hihg enough on result. If coaf = 1, output is (300,300) pixels
    coef = 3

    # Making overlay squared (current overlay is 300*302)
    o_w, o_h = overlay.size
    overlay = overlay.resize((o_w*coef,o_h*coef), Image.Resampling.LANCZOS)

    #New overlay size
    o_w, o_h = overlay.size
    #print(f"New overlay size = {overlay.size}")
    #display(overlay)

    # Downsizing the image
    new_size = (220*coef,220*coef)
    img_resized = img.resize(new_size, Image.Resampling.LANCZOS)
    i_w, i_h = img_resized.size
    #print(img_resized.size)
    #print(i_w, i_h)
    #display(img_resized)


    px_vertical_offset= 1*coef # it seems transparent circle is not perfectly centered
    offset_to_center = ((o_w - i_w) // 2, (o_h - i_h) // 2 + px_vertical_offset)
    #print(((o_w - i_w) % 2 == 0) and ((o_h - i_h) % 2 ==0))

    img_resized_tbg = Image.new('RGBA', (o_w, o_h),
                                    (255, 255, 255,0))
    img_resized_tbg.paste(img_resized, offset_to_center)
    #print(img_resized_tbg.size)

    return Image.alpha_composite(overlay, img_resized_tbg)

In [10]:
path = "/Users/arthurcamu/code/test_vignettes"

circles_path = os.path.join(path, 'cercles')
vignettes_path = os.path.join(path, 'vignettes')

overlay_path = "/Users/arthurcamu/code/"
overlay = Image.open(os.path.join(overlay_path,"masque.png"))

valid_images = [".jpg",".png"]

if not os.path.exists(circles_path):
    os.mkdir(circles_path)

if not os.path.exists(vignettes_path):
    os.mkdir(vignettes_path)

for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    img = Image.open(os.path.join(path,f))
    image_to_circle(img).save(os.path.join(circles_path,f"{f}_circle.png"),"PNG")
    image_to_vignette(img, overlay).save(
        os.path.join(vignettes_path, f"{f}_vignette.png"), "PNG")


# Using OpenCV

In [2]:
import cv2

In [3]:
import matplotlib.pyplot as plt

In [16]:
img1 = cv2.imread('result.png')
img2 = cv2.imread('masque.png')
print(img1.shape,img2.shape)

(800, 800, 3) (302, 300, 3)


In [5]:
img2_r = cv2.resize(img2, (300,300), interpolation=cv2.INTER_AREA)

In [6]:
img2_r.shape[0:2]

(300, 300)

In [7]:
img1_r = cv2.resize(img1, img2_r.shape[0:2], interpolation=cv2.INTER_AREA)

In [8]:
img1_r.shape

(300, 300, 3)

In [10]:
cv2.imshow("img1",img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
new_img = cv2.addWeighted(img1_r, 1, img2_r, 1, 0)

In [14]:
cv2.imwrite('new_blend.png',new_img)

True